In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import cosine

from ase import Atoms, geometry, neighborlist
from ase.io import read, write
from ase.neighborlist import NeighborList
from ase.visualize import view

from scipy import sparse
from scipy.spatial.transform import Rotation

import networkx as nx

import metatensor
from rascaline import SoapPowerSpectrum
from anisoap.representations import EllipsoidalDensityProjection
from anisoap.utils import ClebschGordanReal, cg_combine, standardize_keys

In [2]:
mols = read("planar_mols.xyz", ":")

NLIST_KWARGS = {
    "skin": 0.3,   # doesn't matter for this application.
    "sorted": False,
    "self_interaction": False,
    "bothways": True
}

In [21]:
def build_graph(mol: Atoms, nl=None):
    if nl == None:
        nl = neighborlist.build_neighbor_list(mol, self_interaction=False, bothways=True)
    G = nx.Graph()
    for i in range(len(mol)):
        atom = mol[i]
        nb_indices, offsets = nl.get_neighbors(i)
        nb_atoms = [mol[a] for a in nb_indices]
        el = [(i, nb) for nb in nb_indices]
        G.add_edges_from(el)
        G.nodes[i]["atom"] = mol[i]
    return G

def get_rings(frame, graph=None):
    if graph == None:
        G = build_graph(frame)
    else:
        G = graph
    rings = nx.cycle_basis(G)
    return rings

def get_cluster_data(ring):
    moments, axes = ring.get_moments_of_inertia(vectors=True)
    mass = np.sum([atom.mass for atom in ring])
    E = np.reshape(moments, (3,1))
    coefs = np.array([[0, 1, 1],
                      [1, 0, 1],
                      [1, 1, 0]]) * mass / 5
    return {"axes" : np.sqrt(np.linalg.solve(coefs, E)), "moments" : moments, "eigenvectors" : axes, "mass" : mass}
    
def get_ellipsoids(frame):
    coms = []
    quats = []
    positions = []
    dim1 = []
    dim2 = []
    dim3 = []
    for i, ring in enumerate(get_rings(frame)):
        cluster = frame[[a for a in range(len(frame)) if (a in ring and frame.arrays["numbers"][a] == 6)]]
        dist_vecs, _ = geometry.get_distances(cluster.positions)
        pos_vecs = cluster.positions[0] + dist_vecs[0]
        com = pos_vecs.mean(axis=0)
        data = get_cluster_data(cluster)
        rot = np.asarray(data["eigenvectors"]).T
        if np.isclose(np.linalg.det(rot), -1):
            rot = np.matmul(rot, [[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
        for i in range(3):
            print((data["axes"][i-1]**2 + data["axes"][i-2]**2) * data["mass"] / 5)
            print(data["moments"][i], "\n")
        quat = Rotation.from_matrix(rot).as_quat()
        quat = np.roll(quat, 1)
        quats.append(quat)
        positions.append(pos_vecs)
        coms.append(com)
        dim1.append(data["axes"][0])
        dim2.append(data["axes"][1])
        dim3.append(data["axes"][2])
    ell_frame = Atoms(positions = np.vstack(coms), cell = frame.cell, pbc = frame.pbc)
    ell_frame.arrays["quaternions"] = np.vstack(quats)
    ell_frame.arrays["c_diameter[1]"] = np.array(dim1)
    ell_frame.arrays["c_diameter[2]"] = np.array(dim2)
    ell_frame.arrays["c_diameter[3]"] = np.array(dim3)
    return ell_frame

In [22]:
frame = mols[0]
ells = get_ellipsoids(frame)
write("ells.xyz", ells)
write("frame.xyz", frame)

[71.59301227]
71.59301227328729 

[73.12715094]
73.12715094260395 

[144.71916567]
144.7191656653745 

[72.25355439]
72.25355438976106 

[73.60214674]
73.60214673808392 

[145.85479239]
145.85479238800005 

[72.30011295]
72.30011294721278 

[73.74088246]
73.74088245601457 

[146.04098032]
146.0409803203648 

[72.25883538]
72.25883537958453 

[73.59362651]
73.59362651191798 

[145.85154497]
145.85154497314804 

[71.5893906]
71.58939059818299 

[73.12486506]
73.12486505549741 

[144.71327465]
144.71327465266808 



In [27]:
lmax, nmax, gaussian, cutoff_radius = 9, 6, 1.5, 7.0
mycg = ClebschGordanReal(lmax)

ANISOAP_HYPERS = {
    "max_angular": lmax,
    "max_radial": nmax,
    "radial_basis_name": "gto",
    "rotation_type": "quaternion",
    "rotation_key": "quaternions",
    "radial_gaussian_width": gaussian,
    "cutoff_radius": cutoff_radius,
    "basis_rcond": 1e-8,
    "basis_tol": 1e-4,
}

In [32]:
calculator = EllipsoidalDensityProjection(**ANISOAP_HYPERS)
rep_raw = calculator.transform([ells], show_progress=True)

Computing neighborlist:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.